# Face recognition using neural network features

In this task, you have to construct face recognizer based on features extracted from the neural network. The task consists of two parts: image classification and video classification. In the first one you should classify distinct images and in the second one you will deal with short video sequences.

In [1]:
# !pip install mtcnn
# !conda install scikit-learn

In [2]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras import backend as K

E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [3]:
%pylab inline
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
from copy import copy
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [4]:
import zipfile

def unpack(filename):
    with zipfile.ZipFile(filename) as zf:
        zf.extractall()

In [5]:
# from google.colab import drive
# drive.mount('/content/drive')
assignment_dir = 'face-recognition'

First of all, you have you have to read the data. Run the cell below to unpack data.

In [6]:
# from get_data import unpack
#unpack(assignment_dir + '/Face_Recognition_data.zip')

In [7]:
images_processed=True
videos_processed=True

In [8]:
# !unzip '/content/drive/My Drive/face-recognition/Face_Recognition_data.zip' -d '/content/drive/My Drive/face-recognition'

### Reading data for image and video classification (1 point)

Implement function $\tt{load}$\_$\tt{image}$\_$\tt{data}$. It should return a tuple of 4 dictionaries: the first two are training images and labels, the second two are testing ones. The keys of the dictionaries are the names of the image files and the values are 3-dimensional numpy arrays (for images) or strings with the names (for labels).

$\tt{dir}$\_$\tt{name}$ is the name of directory with data for image classification. If 'Face_Recofnition_data' directory is located in the same directory as this notebook, then the default value can be used.

In [9]:
from skimage import io
import glob
import pandas as pd


def load_image_data(dir_name):
    """Your implementation"""
    images_train={}
    images_test={}
    labels_train={}
    labels_test={}

    for im_path in sorted(glob.glob(dir_name + "/train/images/*.jpg")):
      images_train[im_path.split('/')[-1]] = io.imread(im_path)
    for im_path in sorted(glob.glob(dir_name + "/test/images/*.jpg")):
      images_test[im_path.split('/')[-1]] = io.imread(im_path)  

    labels=pd.read_csv(dir_name+'/train/y_train.csv')
    for rowidx in range(labels.shape[0]):
      labels_train[labels.iloc[rowidx,0]] = labels.iloc[rowidx,1]


    labels=pd.read_csv(dir_name+'/test/y_test.csv')
    for rowidx in range(labels.shape[0]):
      labels_test[labels.iloc[rowidx,0]] = labels.iloc[rowidx,1]
    
    
 
    return images_train, labels_train, images_test, labels_test

In [10]:
if not images_processed:
  x_train, y_train, x_test, y_test = load_image_data(assignment_dir + "/Face_Recognition_data/image_classification")
  np.savez_compressed(assignment_dir+'/loaded_images.npz',x_train=x_train,y_train=y_train,x_test=x_test,y_test=y_test)
else:
  with np.load(assignment_dir + "/loaded_images.npz", allow_pickle=True) as data:
    x_train = data['x_train'].item()
    y_train = data['y_train'].item()
    x_test = data['x_test'].item()
    y_test = data['y_test'].item()
    
print(len(x_train), '\ttraining images')
print(len(x_test), '\ttesting images')

661 	training images
808 	testing images


In [11]:
def visualize(data, labels, function = lambda x:x[0], n_cols = 5, n_rows=1):
    figure(figsize = (3*n_cols,3*n_rows))
    for n,i in enumerate(np.random.choice(list(data.keys()), size = n_cols*n_rows)):
        plt.subplot(n_rows,n_cols,n+1)
        plt.axis('off')
        plt.imshow(function([data[i]]))
        plt.title(labels[i])
    plt.show()

That is how the data looks like.

Let us now read the video classification data, as well. You have to implement function to load video data.

Function $\tt{load}$\_$\tt{video}$\_$\tt{data}$ should also return a tuple of 4 dictionaries: the first two are training images and labels, the second two are testing videos and labels. The training data is in the same format as in image classification task. The keys of testing data and labels are video ids and the values are the lists of frames and the strings with names, respectively.

In [12]:
def load_video_data(dir_name = 'Face_Recognition_data/video_classification'):
    images_train={}
    videos_test={}
    labels_train={}
    labels_test={}

    for im_path in sorted(glob.glob(dir_name + "/train/images/*.jpg")):
      images_train[im_path.split('/')[-1]] = io.imread(im_path)
    
    labels=pd.read_csv(dir_name+'/train/y_train.csv')
    for rowidx in range(labels.shape[0]):
      labels_train[str(labels.iloc[rowidx,0])] = labels.iloc[rowidx,1]

    for im_path in sorted(glob.glob(dir_name + "/test/videos/*/")):
      video_id=im_path.split('/')[-2]
      print(video_id)
      filenames = glob.glob(dir_name + "/test/videos/"+video_id+"/*.jpg")
      sorted_filenames = sorted(filenames, key=lambda x:int(x.split('/')[-1][:-4]))
      videos_test[video_id] = []
      for fn in sorted_filenames:
        videos_test[video_id].append(io.imread(fn))
      

    labels=pd.read_csv(dir_name+'/test/y_test.csv')
    for rowidx in range(labels.shape[0]):
      labels_test[str(labels.iloc[rowidx,0])] = labels.iloc[rowidx,1]

      # images_test[im_path.split('/')[-1]] = io.imread(im_path)  
  

    return images_train, labels_train, videos_test, labels_test

In [13]:
if not videos_processed:
  video_train, train_labels, video_test, test_labels = load_video_data(assignment_dir + "/Face_Recognition_data/video_classification")
  np.savez_compressed(assignment_dir+'/loaded_videos.npz',video_train=video_train, train_labels=train_labels, 
                    video_test=video_test, test_labels=test_labels)
else:
  with np.load(assignment_dir + "/loaded_videos.npz", allow_pickle=True) as data:
    video_train = data['video_train'].item()
    train_labels = data['train_labels'].item()
    video_test = data['video_test'].item()
    test_labels = data['test_labels'].item()  


print(len(video_train), '\ttraining images')
print(len(video_test), '\ttesting videos')

729 	training images
70 	testing videos


### Preprocessing (3 points)
You have to implement preprocessing function in the cell below.
Getting a list of images as an input the this function should detect the face on each image, find the facial keypoints () and then crop and normalize the image according to these keypoints. The output of this function is the list of images which contain only the aligned face and should be converted to the tensor of the shape $(N, 224, 224, 3)\ $ where $N$ is the length of the initial list. You can add extra arguments to the preprocess function if necessary (i.e. flag $\tt{is}$\_$\tt{video}$ to determine if the list of images is video sequence or not).

For face detection and facial keypoint regression you can use your models from the previous tasks.


---
From the above wording of the instructions, we conclude that we are not forced to use the models from previous tasks but can use also other models of our choosing for the pre-processing task.

Exactly this we will do as we anticipate a heap of problems with the previous models, as those models have been trained for fixed resolution and with just small datasets. Instead, we base the pre-processing on face and facial landmark detections of the MTCNN model of the ipazc/MTCNN project (http://github.com/ipazc/mtcnn).


In [14]:
# from week 1 assignment
from skimage import transform

def transform_face(image, eyes, size_multiplier = 2.15):
    e=np.array(eyes)
    center = np.mean(e,axis=0, keepdims=True)
    diff = e[1]-e[0]
    dist=np.sqrt(np.sum(diff*diff))
    angle = np.arctan2(diff[1],diff[0])
    
    img_rotated = transform.rotate(image,np.degrees(angle), center=center)

    #plt.imshow(img_rotated)
    #print('Rotated image')
    #plt.show()

    face_extent = int(size_multiplier*dist) 

    # print('Face extent: ', face_extent)

    box_left = int(center[0,0]-face_extent) 
    box_right = int(box_left + 2* face_extent)

    box_top = int(center[0,1]-face_extent) 
    box_bottom = int(box_top + 2* face_extent)

    pad=max(0,-box_left,-box_top,box_bottom-image.shape[0]-1, box_right-image.shape[1]-1)
    img_crop = np.pad(img_rotated, ((pad,pad),(pad,pad),(0,0)))[box_top+pad:box_bottom+1+pad, box_left+pad:box_right+1+pad]

    BOTTLENECK_SIZE = 90
    img_crop = transform.resize(img_crop, (BOTTLENECK_SIZE, BOTTLENECK_SIZE))
    
    return transform.resize(img_crop, (224, 224))

In [15]:
import scipy.misc
from mtcnn.mtcnn import MTCNN
from matplotlib.patches import Rectangle, Circle

from skimage import exposure
from skimage import filters

import copy

def equalize_color_img(img):
  img_eq = np.zeros_like(img)
  img_eq[:,:,0] = exposure.equalize_hist(img[:,:,0])
  img_eq[:,:,1] = exposure.equalize_hist(img[:,:,1])
  img_eq[:,:,2] = exposure.equalize_hist(img[:,:,2])
  return img_eq

def distance_from_center(img, detection):
    # img is numpay array
    # detection is the face detection object from MTCNN detector

    x, y, width, height = detection['box']
    center_x = (img.shape[1]-1)/2.0
    center_y = (img.shape[0]-1)/2.0
    face_x = x + width/2.0
    face_y = y + height/2.0
    dx = center_x - face_x
    dy = center_y - face_y
    return np.sqrt(dx*dx+dy*dy)

def measure_blur(img):
  return np.var(filters.laplace(img[:,:,0])) + np.var(filters.laplace(img[:,:,1])) + np.var(filters.laplace(img[:,:,2]))

def preprocess_imgs(imgs, size_multiplier=2.15):
    ret_imgs=[]
    detector = MTCNN()
    n_images = len(imgs)
    processed_img = 0

    for img in imgs:
      processed_img += 1
      if processed_img%10 == 1: 
          print ('Preprocessing image {}/{}'.format(processed_img,n_images))
      #plt.imshow(img)
      #plt.title('Source image')
      #plt.show()

      faces = detector.detect_faces(img)
      #for face in faces:
	    #  print(face)
       
      #choose the face that is closest to the image center
      if len(faces) < 1:
        ret_imgs.append(np.zeros((224,224,3)))
        continue

      min_ind=0
      min_dist = distance_from_center(img, faces[0])
      for i in range(1,len(faces)):
        dist = distance_from_center(img, faces[i])
        if dist < min_dist:
          min_ind = i
          min_dist = dist     
       
      #plt.imshow(img)
      #ax = plt.gca()
      face=faces[min_ind]

      #x, y, width, height = face['box']
      #rect = Rectangle((x, y), width, height, fill=False, color='red')
      #ax.add_patch(rect)
      #dot = Circle(face['keypoints']['left_eye'], radius=2, color='green')
      #ax.add_patch(dot)
      #dot = Circle(face['keypoints']['right_eye'], radius=2, color='blue')
      #ax.add_patch(dot)
      #plt.show() 

      if face['confidence']< 0.9:
        ret_imgs.append(np.zeros((224,224,3)))
        continue

      eyes=[face['keypoints']['left_eye'], face['keypoints']['right_eye']]  
      #print(eyes)
      img_transformed = equalize_color_img(transform_face(img, eyes, 
                                                          size_multiplier=size_multiplier))
      
      #blur = np.var(filters.laplace(img_transformed[:,:,0])) + np.var(filters.laplace(img_transformed[:,:,1])) + np.var(filters.laplace(img_transformed[:,:,2]))
      #print('Blur: ',blur)

      img_blurred=copy.deepcopy(img_transformed) 

      #sigma=1
      #for i in range(5):
      #  img_blurred=filters.gaussian(img_blurred)
      #sharpness = measure_blur(img_blurred)
      #MAX_SHARPNESS = 0.0005
      #while sharpness > MAX_SHARPNESS:
      #  img_blurred=filters.gaussian(img_blurred)
      #  sharpness = measure_blur(img_blurred)
        #print('Blur after {} iterations of Gaussian blurring: {}'.format(i+1,blur))


    

      #plt.imshow(img_transformed)
      #plt.show()

      ret_imgs.append(img_blurred)
    return ret_imgs


Using TensorFlow backend.


#### Visualization of preprocessing

The neural network is already trained on the other face dataset. You should use this network as feature extractor to get descriptors of the faces. You can choose any hidden layer you need (or several layers) to extract features and any classification method.

In [16]:
import h5py
from tensorflow.keras.models import load_model
model = load_model(assignment_dir + '/face_recognition_model.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1_1 (Conv2D)             (None, 224, 224, 64)      1792      
_________________________________________________________________
conv1_2 (Conv2D)             (None, 224, 224, 64)      36928     
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 112, 112, 64)      0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2_2 (Conv2D)             (None, 112, 112, 128)     147584    
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 56, 56, 128)       0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 56, 56, 256)       2

Here is an example of using the network as feature extractor. The shape of input tensor has to be (n_images, 224, 224, 3), so you can input several images simultaneously and get their face descriptors of shape (n_images, n_components).

In [17]:
def get_layer_output(images, layer = 'fc7'):
    assert len(images.shape)==4, 'Wrong input dimentionality!'
    assert images.shape[1:]==(224,224,3), 'Wrong input shape!'
    
    layers=layer.split('+')
    n_img = images.shape[0]

    output=np.zeros((images.shape[0],0))

    batch_size = 2
    
    for l in layers: 
      network_output = model.get_layer(l).output
      feature_extraction_model = Model(model.input, network_output)
         
      layer_output=zeros((n_img,feature_extraction_model.predict(images[0,:,:,:].reshape(1,224,224,3)).shape[1]))
    
      
    
      for i in range(int(n_img/batch_size)):
        layer_output[i*batch_size:min(n_img,(i+1)*batch_size),:]=feature_extraction_model.predict(
            images[i*batch_size:min(n_img,(i+1)*batch_size),:,:,:])    
    
      output=np.hstack((output,layer_output))
    return output

In [18]:
img = cv2.resize(x_train['0.jpg'], (224,224)).reshape(1,224,224,3)
out = get_layer_output(img)
print(out.shape)

(1, 4096)


### Training classifier (2 points)


You have to implement class $\tt{Classifier}$ with methods $\tt{fit}$, $\tt{classify}$\_$\tt{images}$ and $\tt{classify}$\_$\tt{videos}$ in the cell below. 
The method $\tt{Classifier.fit}$ gets two dictionaries as input: train images and labels, and trains the classifier to predict the person shown on the image.
$\tt{Classifier.classify}$\_$\tt{images}$ gets the dictionary of test images (with filenames as keys) as input and should return the dictionary of the predicted labels.
$\tt{Classifier.classify}$\_$\tt{videos}$ is similar to the previous one, but gets the dictionary of test videos (with video as keys) as input.

To classify video you can combine the predictions for its frames any way you want (averaging, voting, etc.).
If video classification takes too long you can use face detector not in all the frames but every few frames while preprocessing video frames. 
Besides, sometimes the face is hardly detected on the image and the frame in which the detector works wrong can add noise to the prediction. Hence, the result of the prediction without using such frames may be better.

Now we can build the classifier, fit it and use to predict the labels of testing images.

### Image classification quality (2 points)

Let us check the accuracy of your classification. To obtain 1 point for that rubric your implementation must have accuracy at least 0.90, to obtain 2 points — at least 0.95.

In [19]:
def check_test(output, gt):
    correct = 0.
    total = len(gt)
    for k, v in gt.items():
        if output[k] == v:
            correct += 1
    accuracy = correct / total

    return 'Classification accuracy is %.4f' % accuracy

### Video classification quality (2 points)

Let us check the quality of video classification. To obtain 1 point for that rubric your implementation must have accuracy at least 0.80, to obtain 2 points — at least 0.85.

In [37]:
from sklearn import preprocessing
from sklearn import decomposition
from catboost import CatBoostClassifier, Pool

from skimage.io import imread
import cv2
from os.path import join
class ClassifierVideoCatboost():
    def __init__(self, nn_model):
        self.label_encoder = preprocessing.LabelEncoder()
        self.catboost_model = CatBoostClassifier(verbose=True, iterations=750)
        self.pca1 = decomposition.PCA(0.8)
        self.pca2 = decomposition.PCA(0.8)
        self.pca3 = decomposition.PCA(0.8)
        
        self.layer = 'fc6+fc7+fc8'
        self.size_multiplier=2.15
        self.video_step = 1

    def fit(self, train_imgs, train_labels):
        n_images=len(train_imgs)
        image_counter=0
        
        labels_orig = []
        for id in train_imgs:
          labels_orig.append(train_labels[id])
        
        labels_orig.extend(labels_orig)

        preprocessed_images = np.array(preprocess_imgs(train_imgs.values(), size_multiplier=self.size_multiplier))
        flipped_images=np.flip(preprocessed_images, -2)
        augmented_images = np.vstack((preprocessed_images, flipped_images))

        # augment images with two different scaling bottlenecks

#        bottleneck_70 = np.zeros_like(augmented_images)
#        bottleneck_110 = np.zeros_like(augmented_images)
##
#        n_augm = augmented_images.shape[0]
#        for i in range(n_augm):
#        bottleneck_70[i,:,:] = transform.resize(transform.resize(augmented_images[i], (70, 70)), (224, 224))
#        bottleneck_110[i,:,:] = transform.resize(transform.resize(augmented_images[i], (110, 110)), (224, 224))

#        labels_orig = labels_orig * 3
#        augmented_images = np.vstack((augmented_images, bottleneck_70, bottleneck_110))
        
#        blur1 = np.zeros_like(augmented_images)
#        blur2 = np.zeros_like(augmented_images)

#        n_augm = augmented_images.shape[0]
#        for i in range(n_augm):
#         blur1[i,:,:] = filters.gaussian(augmented_images[i])
#         blur2[i,:,:] = filters.gaussian(blur1[i])
         
#        labels_orig = labels_orig * 3
#        augmented_images = np.vstack((augmented_images, blur1, blur2))

#        print('augmented image set shape {}'.format(augmented_images.shape))

#        #flip the images of the second half horizontally

        pca1 = self.pca1.fit_transform(get_layer_output(augmented_images, layer='fc6'))
        pca2 = self.pca2.fit_transform(get_layer_output(augmented_images, layer='fc7'))
        pca3 = self.pca3.fit_transform(get_layer_output(augmented_images, layer='fc8'))
        # pca2 = self.pca2.fit_transform(get_layer_output(augmented_images,layer='fc7'))
        
        self.x_train_internal= np.hstack((pca1,pca2,pca3))
        self.y_train_internal = self.label_encoder.fit_transform(labels_orig)
        
        self.catboost_model.fit(self.x_train_internal, self.y_train_internal)
        self.fitted=True

    def classify_images(self, test_imgs):
      assert self.fitted

      n_images=len(test_imgs)
      image_counter=0
              
      # x_test= get_layer_output(np.array(preprocess_imgs(test_imgs.values(), size_multiplier=self.size_multiplier)), layer=self.layer)
      preprocess_result = np.array(preprocess_imgs(test_imgs.values(), size_multiplier=self.size_multiplier))   
      pca1 =  self.pca1.transform(get_layer_output(preprocess_result, layer='fc6'))
      pca2 =  self.pca2.transform(get_layer_output(preprocess_result, layer='fc7'))
      pca3 =  self.pca3.transform(get_layer_output(preprocess_result, layer='fc8'))
      x_test = np.hstack((pca1, pca2, pca3))  
      ids=[] 
      for id in test_imgs:
        ids.append(id)
        
      y_test=self.label_encoder.inverse_transform(self.catboost_model.predict(x_test))
      print('mapping predictions back')
      print(y_test.shape)
      for i in range(n_images):
        predictions[ids[i]] = str(y_test[i])
      return predictions
        


      

        
    def classify_videos(self, test_video):
        assert self.fitted
        predictions={}

        video_step = 6

        preprocess_pool=[]
        pool_start={}
        pool_end={}

        # populate pool and preprocess all images in one lump
        for id in test_video:
          pool_start[id] = len(preprocess_pool) 
          preprocess_pool.extend(test_video[id][::video_step])
          pool_end[id] = len(preprocess_pool) 

        preprocess_result = np.array(preprocess_imgs(preprocess_pool, size_multiplier=self.size_multiplier))
        
         
        for id in test_video:
          print('classifying video id ',id )
          pca1 = self.pca1.transform(get_layer_output(preprocess_result[pool_start[id]:pool_end[id]], layer='fc6'))
          pca2 = self.pca2.transform(get_layer_output(preprocess_result[pool_start[id]:pool_end[id]], layer='fc7'))
          pca3 = self.pca3.transform(get_layer_output(preprocess_result[pool_start[id]:pool_end[id]], layer='fc8'))
                                    
          x_pred= np.hstack((pca1, pca2, pca3))
          probs = self.catboost_model.predict_proba(x_pred)
          y_pred= self.label_encoder.inverse_transform(self.catboost_model.predict(x_pred))
          for row in range(probs.shape[0]):
         #   print('Frame ',row)
         #   #best_indices = probs[row,:].argsort()[-5:][::-1]
         #   #print(neigh_dist.shape, neigh_ind.shape)
         #   #print('best indices:', best_indices)
         #   #for i in range(5):
         #   #  print('{} (p={})'.format(self.label_encoder.inverse_transform(best_indices)[i],
         #   #                           probs[row,best_indices[i]]))

#            # collect sliding window of past nearest neighbours
#
#            windowstart=max(0,row-10)
#            ind_window=np.array(neigh_ind[windowstart:row+1]).flatten() 
#            dist_window=np.array(neigh_dist[windowstart:row+1]).flatten() 
#
#            ind_window_sorted = ind_window[np.argsort(dist_window)]
#            dist_window_sorted = np.sort(dist_window)
#
#            
#            print(neigh_dist[row])
#            print(self.label_encoder.inverse_transform(self.y_train_internal[neigh_ind[row]]))
#            print(self.label_encoder.inverse_transform(self.y_train_internal[ind_window_sorted]))
#            print(dist_window_sorted)

            print('Frame prediction: ',y_pred[row])

          lst=list(y_pred)
          predictions[id]=max(set(lst), key=lst.count) # mode of y_pred
          print('prediction for video {}: {}'.format(id,predictions[id]))

        return predictions

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as kNN
from sklearn import preprocessing
from sklearn import decomposition
from statistics import mode

from skimage.io import imread
import cv2
from os.path import join
class ClassifierVideo():
    def __init__(self, nn_model):
        self.label_encoder = preprocessing.LabelEncoder()
        self.standard_scaler = preprocessing.StandardScaler()
        self.fitted = False
        self.feature_dim = 4096
        self.knn_model = kNN(n_neighbors=5)
        self.pca = decomposition.PCA(0.9, whiten=True)
        self.pca2 = decomposition.PCA(0.9, whiten=True)
        self.layer = 'fc7'
        self.size_multiplier=2.15
        self.video_step = 1

    def fit(self, train_imgs, train_labels):
        n_images=len(train_imgs)
        image_counter=0
        
        labels_orig = []
        for id in train_imgs:
          labels_orig.append(train_labels[id])
        
        labels_orig.extend(labels_orig)

        preprocessed_images = np.array(preprocess_imgs(train_imgs.values(), size_multiplier=self.size_multiplier))
        flipped_images=np.flip(preprocessed_images, -2)
        augmented_images = np.vstack((preprocessed_images, flipped_images))

        # augment images with two different scaling bottlenecks

        #bottleneck_70 = np.zeros_like(augmented_images)
        #bottleneck_110 = np.zeros_like(augmented_images)
#
#       n_augm = augmented_images.shape[0]
#        for i in range(n_augm):
#         bottleneck_70[i,:,:] = transform.resize(transform.resize(augmented_images[i], (70, 70)), (224, 224))
#         bottleneck_110[i,:,:] = transform.resize(transform.resize(augmented_images[i], (110, 110)), (224, 224))
#
#        labels_orig = labels_orig * 3
#        augmented_images = np.vstack((augmented_images, bottleneck_70, bottleneck_110))
        
 #       blur1 = np.zeros_like(augmented_images)
 #       blur2 = np.zeros_like(augmented_images)

 #       n_augm = augmented_images.shape[0]
 #       for i in range(n_augm):
 #        blur1[i,:,:] = filters.gaussian(augmented_images[i])
 #        blur2[i,:,:] = filters.gaussian(blur1[i])
         
 #       labels_orig = labels_orig * 3
 #       augmented_images = np.vstack((augmented_images, blur1, blur2))

        print('augmented image set shape {}'.format(augmented_images.shape))

        #flip the images of the second half horizontally

        pca1 = self.pca.fit_transform(get_layer_output(augmented_images,layer=self.layer))
        # pca2 = self.pca2.fit_transform(get_layer_output(augmented_images,layer='fc7'))
        
        self.x_train_internal= pca1
        self.y_train_internal = self.label_encoder.fit_transform(labels_orig)
        
        self.knn_model.fit(self.x_train_internal, self.y_train_internal)
        self.fitted=True

    def classify_images(self, test_imgs):
      assert self.fitted
      predictions={}

      n_images=len(test_imgs)
      image_counter=0
              
      x_test= self.pca.transform(get_layer_output(np.array(preprocess_imgs(test_imgs.values(), size_multiplier=self.size_multiplier)), layer=self.layer))
      ids=[] 
      for id in test_imgs:
        ids.append(id)

      y_test=self.label_encoder.inverse_transform(self.knn_model.predict(x_test))
      print('mapping predictions back')
      print(y_test.shape)
      for i in range(n_images):
        predictions[ids[i]] = str(y_test[i])
      return predictions
        


      

        
    def classify_videos(self, test_video):
        assert self.fitted
        predictions={}

        video_step = 6

        preprocess_pool=[]
        pool_start={}
        pool_end={}

        # populate pool and preprocess all images in one lump
        for id in test_video:
          pool_start[id] = len(preprocess_pool) 
          preprocess_pool.extend(test_video[id][::video_step])
          pool_end[id] = len(preprocess_pool) 

        preprocess_result = np.array(preprocess_imgs(preprocess_pool, size_multiplier=self.size_multiplier))
        
         
        for id in test_video:
          print('classifying video id ',id )
          pca1 = self.pca.transform(get_layer_output(preprocess_result[pool_start[id]:pool_end[id]], layer=self.layer))
          # pca2 = self.pca2.transform(get_layer_output(preprocess_result[pool_start[id]:pool_end[id]], layer='fc7'))
                                    
          x_pred= pca1 # np.hstack((pca1, pca2))
          probs = self.knn_model.predict_proba(x_pred)
          y_pred= self.label_encoder.inverse_transform(self.knn_model.predict(x_pred))
 #         neigh_dist, neigh_ind = self.knn_model.kneighbors(x_pred, return_distance=True)
          for row in range(probs.shape[0]):
         #   print('Frame ',row)
         #   #best_indices = probs[row,:].argsort()[-5:][::-1]
         #   #print(neigh_dist.shape, neigh_ind.shape)
         #   #print('best indices:', best_indices)
         #   #for i in range(5):
         #   #  print('{} (p={})'.format(self.label_encoder.inverse_transform(best_indices)[i],
         #   #                           probs[row,best_indices[i]]))

#            # collect sliding window of past nearest neighbours
#
#            windowstart=max(0,row-10)
#            ind_window=np.array(neigh_ind[windowstart:row+1]).flatten() 
#            dist_window=np.array(neigh_dist[windowstart:row+1]).flatten() 
#
#            ind_window_sorted = ind_window[np.argsort(dist_window)]
#            dist_window_sorted = np.sort(dist_window)
#
#            
#            print(neigh_dist[row])
#            print(self.label_encoder.inverse_transform(self.y_train_internal[neigh_ind[row]]))
#            print(self.label_encoder.inverse_transform(self.y_train_internal[ind_window_sorted]))
#            print(dist_window_sorted)

            print('Frame prediction: ',y_pred[row])

          lst=list(y_pred)
          predictions[id]=max(set(lst), key=lst.count) # mode of y_pred
          print('prediction for video {}: {}'.format(id,predictions[id]))

        return predictions

In [ ]:
len(video_test['0'][::2])


In [ ]:
# %%capture cap_out --no-stderr

def extract_subdict(d, keys):
    return dict((k, d[k]) for k in keys if k in d)

difficult_labels=['10','14','16','29','35','40','43','49','5','50','53','54','57','59','63']
testing_labels = ['0','1','11']+difficult_labels
video_classifier = ClassifierVideo(model)
video_classifier.fit(video_train, train_labels)
y_video_difficult_out = video_classifier.classify_videos(extract_subdict(video_test, testing_labels))

In [ ]:
with open('fc7-heq-2.15.txt', 'w') as f:
    f.write(cap_out.stdout)

In [38]:
# %%capture cap_out --no-stderr

video_train_aug=video_train
train_labels_aug=train_labels

for lbl in x_train:
    video_train_aug['img:'+lbl] = x_train[lbl]
    train_labels_aug['img:'+lbl] = y_train[lbl]

video_classifier = ClassifierVideoCatboost(model)
#video_classifier.fit(video_train, train_labels)
video_classifier.fit(video_train_aug, train_labels_aug)
y_video_out = video_classifier.classify_videos(video_test)

print(check_test(y_video_out, test_labels))


Preprocessing image 1/1390
Preprocessing image 11/1390
Preprocessing image 21/1390
Preprocessing image 31/1390
Preprocessing image 41/1390
Preprocessing image 51/1390
Preprocessing image 61/1390
Preprocessing image 71/1390
Preprocessing image 81/1390
Preprocessing image 91/1390
Preprocessing image 101/1390
Preprocessing image 111/1390
Preprocessing image 121/1390
Preprocessing image 131/1390
Preprocessing image 141/1390
Preprocessing image 151/1390
Preprocessing image 161/1390
Preprocessing image 171/1390
Preprocessing image 181/1390
Preprocessing image 191/1390
Preprocessing image 201/1390
Preprocessing image 211/1390
Preprocessing image 221/1390
Preprocessing image 231/1390
Preprocessing image 241/1390
Preprocessing image 251/1390
Preprocessing image 261/1390
Preprocessing image 271/1390
Preprocessing image 281/1390
Preprocessing image 291/1390
Preprocessing image 301/1390
Preprocessing image 311/1390
Preprocessing image 321/1390
Preprocessing image 331/1390
Preprocessing image 341/1

E:\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Nancy_Pelosi
Frame prediction:  Bill_Simon
prediction for video 0: Nancy_Pelosi
classifying video id  1


E:\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Paul_Wolfowitz
Frame prediction:  Bill_Simon
prediction for video 1: Paul_Wolfowitz
classifying video id  10


...

E:\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Eduard_Shevardnadze
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Mahmoud_Abbas
Frame prediction:  Bill_Simon
prediction for video 7: Mahmoud_Abbas
classifying video id  8


E:\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Rick_Perry
Frame prediction:  Tim_Henman
Frame prediction:  Rick_Perry
Frame prediction:  Bill_Simon
prediction for video 8: Rick_Perry
classifying video id  9
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Heston
Frame prediction:  Charlton_Hes

E:\Anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_label.py:289: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# %%capture cap_out --no-stderr

video_train_aug=video_train
train_labels_aug=train_labels

for lbl in x_train:
    video_train_aug['img:'+lbl] = x_train[lbl]
    train_labels_aug['img:'+lbl] = y_train[lbl]

video_classifier = ClassifierVideo(model)
#video_classifier.fit(video_train, train_labels)
video_classifier.fit(video_train_aug, train_labels_aug)
y_video_out = video_classifier.classify_videos(video_test)

print(check_test(y_video_out, test_labels))


In [ ]:
y_video_out = video_classifier.classify_videos(video_test)

print(check_test(y_video_out, test_labels))

In [ ]:
with open('fc7-heq-bottleneck-2.15-all.txt', 'w') as f:
    f.write(cap_out.stdout)

In [ ]:
print(check_test(y_video_out, test_labels))

In [ ]:
video_classifier.pca.n_components_

In [ ]:
img_classifier = ClassifierVideo(model)
img_classifier.fit(x_train, y_train)
y_out = img_classifier.classify_images(x_test)
print(check_test(y_out, y_test))